# Setup Environment

In [3]:
!conda install -c conda-forge BeautifulSoup4 --yes 
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # import k-means from clustering stage

import requests #import requests

from bs4 import BeautifulSoup #import BeautifulSoup

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.2            |           py36_0          59 KB  conda-forge
    beautifulsoup4-4.8.0       |           py36_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         202 KB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.2-py36_0 conda-forge

The following packages will be UPDATED:

    beautifulsoup4: 4.6.3-py37_0             --> 4.8.0-py36_0 conda-forge


soupsieve-1.9.2     

In [4]:
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Scraping Postcode, Borough and Neighbourhood information from Wikipedia

In [5]:
# Use BeautifulSoup to scrape
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')


# Get the sort table
sorttable=soup.find('table', attrs={'class':'wikitable sortable'})

# Name of the columns
columns = ['Postcode','Borough','Neighbourhood']

# Get all lines except the first line
lines=sorttable.find_all('tr')
lines=lines[1:len(lines)]

# Format the lines
for i, line in enumerate(lines): lines[i] = str(lines[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")
    
# Create the dataframe
df=pd.DataFrame(lines)
df[columns] = df[0].str.split("</td>\n<td>", n = 2, expand = True) 
df.drop(columns=[0],inplace=True)

#Drop the lines with unassigned borough
df = df.drop(df[(df.Borough == "Not assigned")].index)

# Rename the Neighbourhood to the name of Borough if it is 'Not assigned'
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)

# extract titles from columns
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

df.update(
    df.Borough.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

# delete Toronto annotation 
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto",""))
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))
df.update(
    df.Borough.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto",""))
df.update(
    df.Borough.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))

# combine multiple neighborhoods with the same post code
df2 = pd.DataFrame({'Postcode':df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes

df2.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront , Regent Park"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


# Adding Geo-spatial data

In [6]:
#add Geo-spatial data
df3= pd.read_csv("http://cocl.us/Geospatial_data")
df3.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df3.set_index("Postcode")
df2.set_index("Postcode")
df_trt=pd.merge(df2, df3)
df_trt.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront , Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [7]:
# Get the geographical coordinates of Toronto
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="traveler")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON, Canada are 43.653963, -79.387207.


In [8]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_trt['Latitude'], df_trt['Longitude'], df_trt['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Get Data from FourSquare

In [9]:
CLIENT_ID = 'H0XDGWHBYKRTY5GUFIAPDMJNAOHCJMIUQL3CJ3SEM2CJXENZ' # your Foursquare ID
CLIENT_SECRET = 'V1B32VSYNPGWELBXIA3LJUQDJSDBKLGE54KRMRKAGWAIY2IY' # your Foursquare Secret
VERSION = '20190722' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API

Create a function to return the unique categories for each neighbourhood

In [10]:
def exploreNbhd(names, latitudes, longitudes, radius=700):
    
    venues=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue in venues for item in venue])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get venues for each of the neighbourhood

In [12]:
venues = exploreNbhd(names=df_trt['Neighborhood'],
                                   latitudes=df_trt['Latitude'],
                                   longitudes=df_trt['Longitude']
                                  )
venues.head(10)

Parkwoods
Victoria Village
Harbourfront , Regent Park
Lawrence Manor, Lawrence Heights
Queen's Park 
Islington Avenue
Malvern, Rouge
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Princess Gardens, West Deane Park, Martin Grove, Islington, Cloverdale
Highland Creek , Rouge Hill, Port Union
Don Mills South, Flemingdon Park
Woodbine Heights
St. James Town
Humewood-Cedarvale
Markland Wood, Old Burnhamthorpe, Bloordale Gardens, Eringate
Morningside, Guildwood, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Downsview North, Wilson Heights, Bathurst Manor
Thorncliffe Park
Adelaide, Richmond, King
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Oriole, Henry Farm
Northwood Park, York University
East Toronto
Union Station , Harbourfront East, Toronto Islands
Little Portugal, Trinity–Bellwoods
Ionview, Kennedy Park, East Birchmount Park
Bayview Village
CFB Toronto, Downsv

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,PetSmart,43.748639,-79.333488,Pet Store
2,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
3,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Parkwoods,43.753259,-79.329656,Ranchdale Park,43.751388,-79.322138,Park
5,Parkwoods,43.753259,-79.329656,Joey,43.753441,-79.321640,Burger Joint
6,Parkwoods,43.753259,-79.329656,Three Valleys Park,43.751195,-79.337356,Park
7,Parkwoods,43.753259,-79.329656,Splash pad @ Brookbanks park,43.759167,-79.331294,Playground
8,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
9,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [13]:
# Drop redundant columns
venues_cat = venues.drop(['Venue','Venue Longitude','Venue Latitude'],axis=1,inplace=False)
# Rename all kinds of Asian restaurant to have the same name 'Asian Restaurant'
venues_cat.loc[(venues_cat['Venue Category'].isin(['Chinese Restaurant','Japanese Restaurant','Thai Restaurant','Korean Restaurant','Ramen Restaurant','Sushi Restaurant','Asian Restaurant','Dim Sum Restaurant','Cantonese Restaurant','Shanghai Restaurant','Dumpling Restaurant','Taiwanese Restaurant','Vietnamese Restaurant','Indian Chinese Restaurant'])) , 'Venue Category'] = 'Asian Restaurant'

# Calculate the total count of venues for each Neighbourhood
venues_total = venues_cat.groupby(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude']).count().rename(columns={'Venue Category':'Total Count'}).reset_index()

# Caculate the count of Asian restaurant for each Neighbourhood
venues_asian = venues_cat[venues_cat['Venue Category'].isin(['Chinese Restaurant','Japanese Restaurant','Thai Restaurant','Korean Restaurant','Ramen Restaurant','Sushi Restaurant','Asian Restaurant','Dim Sum Restaurant'])]
venues_asian_total = venues_asian.groupby('Neighborhood').count().rename(columns={'Venue Category':'Asian Rstrt Count'}).reset_index().drop(columns=['Neighborhood Latitude','Neighborhood Longitude'])

# Left join the two dataframes to calculate the ratio of Asian restaurant for each neighbourhood
venues_r = venues_total.merge(venues_asian_total,how='left',on='Neighborhood').fillna(0)
venues_r['Ratio'] = venues_r['Asian Rstrt Count']/venues_r['Total Count']

# Drop redundant columns
venues_r.drop(['Total Count','Asian Rstrt Count'],axis=1,inplace=True)
venues_r.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Ratio
0,"Adelaide, Richmond, King",43.650571,-79.384568,0.090000
1,Agincourt,43.794200,-79.262029,0.100000
2,"Agincourt North, Steeles East, L'Amoreaux East...",43.815252,-79.284577,0.117647
3,"Alderwood, Long Branch",43.602414,-79.543484,0.000000
4,Bayview Village,43.786947,-79.385975,0.250000


# Segmenting and Clustering Neighborhoods
    Prepare the dataframe to be used for segmenting and clustering. We are going to clustering the data by 1) ratio of Asian Restaurant among all venues, and 2) Top 5 venues from each neighborhood, so we need to get the data ready according to that.
    Calculate the ratio of Asian Restaurant


In [14]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_cat[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = venues_cat['Neighborhood'] 


toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Castle,Cheese Shop,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Plane,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Swim School,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Theater,Theme Park,Theme Restaura

In [19]:
# Sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
# Create the new dataframe and display the top 5 venues for each neighborhood.
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, Richmond, King",Asian Restaurant,Coffee Shop,Café,Bar,Steakhouse
1,Agincourt,Asian Restaurant,Skating Rink,Sandwich Place,Restaurant,Badminton Court
2,"Agincourt North, Steeles East, L'Amoreaux East...",Pizza Place,Asian Restaurant,Fast Food Restaurant,Noodle House,BBQ Joint
3,"Alderwood, Long Branch",Pizza Place,Convenience Store,Pharmacy,Gas Station,Skating Rink
4,Bayview Village,Asian Restaurant,Bank,Grocery Store,Café,Skating Rink


Merge the 2 dataframes

In [21]:
venues_merged = venues_r.merge(neighborhoods_venues_sorted,how='left',on='Neighborhood').set_index('Neighborhood',inplace=False)
venues_merged.head()

,Neighborhood Latitude,Neighborhood Longitude,Ratio,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
Neighborhood,,,,,,,,
"Adelaide, Richmond, King",43.650571,-79.384568,0.090000,Asian Restaurant,Coffee Shop,Café,Bar,Steakhouse
Agincourt,43.794200,-79.262029,0.100000,Asian Restaurant,Skating Rink,Sandwich Place,Restaurant,Badminton Court
"Agincourt North, Steeles East, L'Amoreaux East, Milliken, Ontario",43.815252,-79.284577,0.117647,Pizza Place,Asian Restaurant,Fast Food Restaurant,Noodle House,BBQ Joint
"Alderwood, Long Branch",43.602414,-79.543484,0.000000,Pizza Place,Convenience Store,Pharmacy,Gas Station,Skating Rink
Bayview Village,43.786947,-79.385975,0.250000,Asian Restaurant,Bank,Grocery Store,Café,Skating Rink


In [22]:
# Get the unique category names
cat_unique=venues['Venue Category'].unique()
# Create a dictionary for the mapping
dictionary = dict(zip(list(cat_unique), list(range(len(cat_unique)))))

# Mapping names of venue category into corresponding numbers
venues_clst=venues_merged.drop(columns=['Neighborhood Latitude','Neighborhood Longitude'],axis=1,inplace=False)
venues_clst['1st Most Common Venue'] = venues_clst['1st Most Common Venue'].replace(dictionary)
venues_clst['2nd Most Common Venue'] = venues_clst['2nd Most Common Venue'].replace(dictionary)
venues_clst['3rd Most Common Venue'] = venues_clst['3rd Most Common Venue'].replace(dictionary)
venues_clst['4th Most Common Venue'] = venues_clst['4th Most Common Venue'].replace(dictionary)
venues_clst['5th Most Common Venue'] = venues_clst['5th Most Common Venue'].replace(dictionary)

venues_clst.head()

,Ratio,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
Neighborhood,,,,,,
"Adelaide, Richmond, King",0.090000,120,7,25,42,106
Agincourt,0.100000,120,95,79,14,273
"Agincourt North, Steeles East, L'Amoreaux East, Milliken, Ontario",0.117647,9,120,2,189,131
"Alderwood, Long Branch",0.000000,9,124,46,260,95
Bayview Village,0.250000,120,45,57,25,95


As we can not apply k-means to process float and string data at the same time, we need to convert the names of the venue category (string) into numbers

In [23]:
# set number of clusters
kclusters = 10

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_clst)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 8, 7, 7, 2, 9, 4, 7, 4, 0], dtype=int32)

OK, we can go ahead and clustering neighborhoods now

In [24]:
# add clustering labels
venues_merged.insert(0, 'Cluster Labels', kmeans.labels_)
venues_merged.reset_index(inplace=True)
venues_merged

,Neighborhood,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Ratio,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, Richmond, King",2,43.650571,-79.384568,0.090000,Asian Restaurant,Coffee Shop,Café,Bar,Steakhouse
1,Agincourt,8,43.794200,-79.262029,0.100000,Asian Restaurant,Skating Rink,Sandwich Place,Restaurant,Badminton Court
2,"Agincourt North, Steeles East, L'Amoreaux East...",7,43.815252,-79.284577,0.117647,Pizza Place,Asian Restaurant,Fast Food Restaurant,Noodle House,BBQ Joint
3,"Alderwood, Long Branch",7,43.602414,-79.543484,0.000000,Pizza Place,Convenience Store,Pharmacy,Gas Station,Skating Rink
4,Bayview Village,2,43.786947,-79.385975,0.250000,Asian Restaurant,Bank,Grocery Store,Café,Skating Rink
5,"Bedford Park, Lawrence Manor East",9,43.733283,-79.419750,0.068966,Coffee Shop,Italian Restaurant,Asian Restaurant,Greek Restaurant,Sandwich Place
6,Berczy Park,4,43.644771,-79.373306,0.030000,Coffee Shop,Café,Hotel,Restaurant,Beer Bar
7,Business Reply Mail Processing Centre 969 Eastern,7,43.662744,-79.321558,0.022727,Fast Food Restaurant,Light Rail Station,Park,Burrito Place,Brewery
8,"CFB Toronto, Downsview East",4,43.737473,-79.464763,0.000000,Coffee Shop,Sandwich Place,Airport,Park,Dive Bar
9,Caledonia-Fairbanks,0,43.689026,-79.453512,0.076923,Park,Gym,Asian Restaurant,Grocery Store,Mexican Restaurant


Finally, let's visualize the resulting clusters

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(venues_merged['Neighborhood Latitude'], venues_merged['Neighborhood Longitude'], venues_merged['Neighborhood'], venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters